# Layers

GDS layers have 2 integer numbers: GDSlayer, GDSpurpose

Layers also need to be represented with a color and a level of opacity when showing the circuits.

In [ ]:
import dataclasses
import pp
from pp.layers import preview_layerset

In [ ]:
layer_wg = pp.LAYER.WG
print(layer_wg)

In [ ]:
c = preview_layerset()
c

As you can see in the [UBC PDK](https://github.com/gdsfactory/ubc) you can combine all your layers in a dataclass.

In [ ]:
@dataclasses.dataclass
class Layer:
    WG = (1, 0)
    DEVREC = (68, 0)
    LABEL = (10, 0)
    PORT = (1, 10)
    FLOORPLAN = (99, 0)
    WGN = (34, 0)
    WGN_CLAD = (36, 0)


LAYER = Layer()
port_layer2type = {LAYER.PORT: "optical"}
port_type2layer = {v: k for k, v in port_layer2type.items()}

LAYER.DEVREC

As you can see in `pdk.tech` you can also define all your technology constants in a dataclass

In [ ]:
from typing import Tuple
from pp.tech import Tech, TechNitrideCband

In [ ]:
@dataclasses.dataclass(frozen=True)
class TechNitrideOband(TechNitrideCband):
    name: str = "nitride_oband"
    wg_width: float = 1.0
    bend_radius: float = 20.0
    cladding_offset: float = 3.0
    layer_wg: Layer = LAYER.WGN
    layers_cladding: Tuple[Layer, ...] = (LAYER.WGN_CLAD,)
    layer_label: Layer = LAYER.LABEL
    taper_length: float = 20.0
    taper_width: float = 1.0


In [ ]:
TECH_NITRIDE_O = TechNitrideOband()

In [ ]:
TECH_NITRIDE_O

How do you customize components with a particular technology?

You can create a `Pdk` with the custom `tech` technology.

In [ ]:
from pp.pdk import PdkNitrideCband

@dataclasses.dataclass
class PdkNitrideOband(PdkNitrideCband):
    tech: Tech = TECH_NITRIDE_O

PDK_NITRIDE_O = PdkNitrideOband()

In [ ]:
p = PDK_NITRIDE_O

In [ ]:
c = p.ring_single()
c

In [ ]:
cc = p.add_fiber_array(c, optical_routing_type=0, with_align_ports=False)
cc

You can also mix elements from different PDKs

In [ ]:
from pp.pdk import PDK_METAL1

In [ ]:
c = pp.Component('rings_from_different_pdks')
ring_metal = PDK_METAL1.ring_single()
ring_nitride = PDK_NITRIDE_O.ring_single()

ring_nitride_ref = c.add_ref(ring_nitride)
ring_metal_ref = c.add_ref(ring_metal)
ring_metal_ref.ymin = ring_nitride.ymax + 10
c